In [1]:
import geopandas as gp

In [ ]:
#!wget https://www.bcn.cl/siit/obtienearchivo?id=repositorio/10221/10396/1/division_comunal.zip

In [6]:
df = gp.read_file('../data/division_comunal/division_comunal.shp')
df_rm = df[df.NOM_REG=='Región Metropolitana de Santiago']
df_rm.to_file("../data/division_comunal.geojson", driver="GeoJSON")